# Avaliação do Modelo WDO
Avaliação completa do modelo com backtest, métricas de performance e análise de risco.

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path(os.path.dirname(os.getcwd()))
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Configura visualizações
plt.style.use('seaborn')
pd.set_option('display.max_columns', None)

## 1. Carrega Dados e Modelo

In [ ]:
# Carrega dados
from src.data.loaders.market_data_loader import MarketDataLoader

db_path = PROJECT_ROOT / 'src' / 'data' / 'database' / 'candles.db'
loader = MarketDataLoader(str(db_path))
data = loader.load_data()

print(f'Período dos dados: {data.index.min()} até {data.index.max()}')

## 2. Backtest

In [ ]:
from src.ml.features.feature_engineering import FeatureEngineering
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent
from src.agents.agent_coordinator import AgentCoordinator

# Prepara features
engineer = FeatureEngineering()
features = engineer.create_technical_features(data)
features = engineer.create_temporal_features(features)

# Configura agentes
risk_agent = RiskManagementAgent(max_drawdown=-0.05)
regime_agent = MarketRegimeAgent(window_size=20)

coordinator = AgentCoordinator([risk_agent, regime_agent])

# Prepara dados para backtest
test_size = int(len(features) * 0.3)  # 30% para teste
train_data = features[:-test_size]
test_data = features[-test_size:]

print(f'Dados de treino: {len(train_data)} registros')
print(f'Dados de teste: {len(test_data)} registros')

In [ ]:
from src.backtest.monte_carlo import MonteCarloSimulator

# Executa simulações
simulator = MonteCarloSimulator(n_simulations=1000)

# Simula com dados de teste
returns = test_data['returns'].dropna()
simulation = simulator.simulate_returns(
    historical_returns=returns,
    initial_capital=100000
)

print('
Resultados da Simulação:')
print(f'VaR (95%): {simulation.var:.2%}')
print(f'CVaR (95%): {simulation.cvar:.2%}')

# Plota resultados
simulator.plot_simulations(simulation)

## 3. Métricas de Performance

In [ ]:
from src.reporting.summary.performance_summary import PerformanceSummary

# Calcula métricas
summary = PerformanceSummary()
metrics = summary.calculate_metrics(
    trades=simulation.trades
)

print('
Métricas de Performance:')
for metric, value in metrics.items():
    if isinstance(value, float):
        print(f'{metric}: {value:.2%}')
    else:
        print(f'{metric}: {value}')

## 4. Análise de Risco

In [ ]:
# Calcula drawdown
equity = simulation.equity_curves.mean(axis=0)
running_max = np.maximum.accumulate(equity)
drawdown = (equity - running_max) / running_max * 100

plt.figure(figsize=(12, 6))
plt.plot(drawdown)
plt.title('Drawdown ao Longo do Tempo')
plt.xlabel('Dias')
plt.ylabel('Drawdown (%)')
plt.grid(True)
plt.show()

print(f'Máximo Drawdown: {drawdown.min():.2f}%')

## 5. Análise dos Agentes

In [ ]:
# Coleta insights dos agentes
insights = coordinator.get_agent_insights()

print('
Insights dos Agentes:')
for agent_name, agent_insights in insights.items():
    print(f'
{agent_name}:')
    for metric, value in agent_insights.items():
        print(f'- {metric}: {value}')